<h4>Versions of python packages</h4>  <br>json = 2.0.9  <br>csv = 1.0  <br>pd = 1.01  <br>re = 2.2.1  <br>nltk = 3.4.5  <br>sklearn = 0.22.1  <br>nkeras = 2.31  

In [556]:
import json 
import csv 
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import keras
from keras.models import Sequential
from keras import layers

In [557]:
#The file path need to be changed to train.json file path of host machine
with open(r'/Users/varunsai/Desktop/NLP/project-files/train.json') as json_file: 
    data = json.load(json_file) 

In [558]:
text_data=data.values()

In [559]:
textual_data=[]
labels=[]
for i in range(len(list(text_data))):
    textual_data.append(list(text_data)[i]['text'])
    labels.append(list(text_data)[i]['label'])

In [560]:
training_data=pd.DataFrame(list(zip(textual_data, labels)), 
               columns =['text', 'label'])

In [561]:
training_data.label.value_counts()

1    1168
Name: label, dtype: int64

In [562]:
training_data.head()

,text,label
0,why houston flooding isn‘t a sign of climate c...,1
1,The U.N. Intergovernmental Panel on Climate Ch...,1
2,Bureau Now Sets Strict Limits on Cooling\nOVER...,1
3,The Dirty Extractive Underbelly of Clean Energ...,1
4,why climate change seems to have faded from th...,1


In [563]:
#The file path need to be changed to dev.json file path of host machine
with open(r'/Users/varunsai/Desktop/NLP/project-files/dev.json') as json_file: 
    data = json.load(json_file) 

In [564]:
text_data=data.values()

textual_data=[]
labels=[]
for i in range(len(list(text_data))):
    textual_data.append(list(text_data)[i]['text'])
    labels.append(list(text_data)[i]['label'])

dev_data=pd.DataFrame(list(zip(textual_data, labels)), 
               columns =['text', 'label'])

In [565]:
#The file path need to be changed to test-unlabelled.json file path of host machine
with open(r'/Users/varunsai/Desktop/NLP/project-files/test-unlabelled.json') as json_file: 
    data = json.load(json_file) 

text_data=data.values()

textual_data=[]
for i in range(len(list(text_data))):
    textual_data.append(list(text_data)[i]['text'])
    

test_data=pd.DataFrame(textual_data, 
               columns =['text'])

In [566]:
#The file path need to be changed to negative_label.json file path of host machine
negative_label_data=pd.read_json(r'/Users/varunsai/Desktop/NLP/Final_files/NLP/negative_label.json')

In [567]:
negative_label_data.head()

,text,label
0,to over 200 countries attending crucial climat...,0
1,round of coral bleaching. the greenhouse gas e...,0
2,"if it ends up in landfill, its carbon footprin...",0
3,perpetrators and caps on prices if they are ne...,0
4,have come together once again at the un climat...,0


In [568]:
training_mds=negative_label_data.append(training_data)

In [569]:
## data preprocessing
training_mds['text']=training_mds['text'].apply(lambda x:x.lower())
test_data['text']=test_data['text'].apply(lambda x:x.lower())
dev_data['text']=dev_data['text'].apply(lambda x:x.lower())

In [570]:
training_mds['cleaned_text']=training_mds['text'].apply(lambda x:re.sub('[^a-zA-Z]',' ',x))
test_data['cleaned_text']=test_data['text'].apply(lambda x:re.sub('[^a-zA-Z]',' ',x))
dev_data['cleaned_text']=dev_data['text'].apply(lambda x:re.sub('[^a-zA-Z]',' ',x))

In [571]:
stopwords=list(set(stopwords.words('english')))

In [572]:
training_mds['tokenized_word']=training_mds['cleaned_text'].apply(lambda x:word_tokenize(x))
test_data['tokenized_word']=test_data['cleaned_text'].apply(lambda x:word_tokenize(x))
dev_data['tokenized_word']=dev_data['cleaned_text'].apply(lambda x:word_tokenize(x))

In [573]:
#Stemming and Lemmatisation

from nltk.stem import WordNetLemmatizer

ps = PorterStemmer()
training_mds['tokenized_word']=training_mds['tokenized_word'].apply(lambda x: [ps.stem(i) for i in x if not i in stopwords])
test_data['tokenized_word']=test_data['tokenized_word'].apply(lambda x: [ps.stem(i) for i in x if not i in stopwords])
dev_data['tokenized_word']=dev_data['tokenized_word'].apply(lambda x: [ps.stem(i) for i in x if not i in stopwords])

ps = WordNetLemmatizer()
training_mds['tokenized_word']=training_mds['tokenized_word'].apply(lambda x: [ps.lemmatize(i) for i in x if not i in stopwords])
test_data['tokenized_word']=test_data['tokenized_word'].apply(lambda x: [ps.lemmatize(i) for i in x if not i in stopwords])
dev_data['tokenized_word']=dev_data['tokenized_word'].apply(lambda x: [ps.lemmatize(i) for i in x if not i in stopwords])

In [574]:
training_mds['processed_text_data']=training_mds['tokenized_word'].apply(lambda x:' '.join(x))
test_data['processed_text_data']=test_data['tokenized_word'].apply(lambda x:' '.join(x))
dev_data['processed_text_data']=dev_data['tokenized_word'].apply(lambda x:' '.join(x))

In [575]:
#Countvectorizer

cv=CountVectorizer()

In [576]:
train_x = training_mds['processed_text_data']
train_y = training_mds['label']
test_x = test_data['processed_text_data']

In [577]:
training_data_initial=cv.fit_transform(train_x)
testing_data_initial=cv.transform(test_x)
dev_data_final=cv.transform(dev_data['processed_text_data'])

In [578]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

In [579]:
## Fitting logistic regression

In [580]:
lr=LogisticRegression()
lr.fit(training_data_initial,train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [581]:
logreg_test_prediction=lr.predict(testing_data_initial)

In [582]:
logreg_test_prediction=pd.DataFrame(logreg_test_prediction)

In [583]:
logreg_test_prediction.iloc[:,0].value_counts()

1    1185
0     225
Name: 0, dtype: int64

In [584]:
## fitting on validation data
prediction_validation=lr.predict(dev_data_final)

In [585]:
logreg_results = classification_report(prediction_validation, dev_data['label'])
print(logreg_results)
confusion_matrix(prediction_validation,dev_data['label'])

              precision    recall  f1-score   support

           0       0.10      0.71      0.18         7
           1       0.96      0.52      0.67        93

    accuracy                           0.53       100
   macro avg       0.53      0.62      0.42       100
weighted avg       0.90      0.53      0.64       100



array([[ 5,  2],
       [45, 48]])

In [586]:
## Fitting SVM

In [587]:
from sklearn.svm import SVC

In [588]:
svm=SVC(kernel='sigmoid')
svm.fit(training_data_initial,train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [589]:
svm_test_prediction=svm.predict(testing_data_initial)

In [590]:
svm_test_prediction=pd.DataFrame(svm_test_prediction)

In [591]:
svm_test_prediction.iloc[:,0].value_counts()

1    1201
0     209
Name: 0, dtype: int64

In [592]:
## fitting on validation data
prediction_validation=svm.predict(dev_data_final)

In [593]:
svm_results = classification_report(prediction_validation, dev_data['label'])
print(svm_results)
confusion_matrix(prediction_validation,dev_data['label'])

              precision    recall  f1-score   support

           0       0.16      0.89      0.27         9
           1       0.98      0.54      0.70        91

    accuracy                           0.57       100
   macro avg       0.57      0.71      0.48       100
weighted avg       0.91      0.57      0.66       100



array([[ 8,  1],
       [42, 49]])

In [594]:
svm_test_prediction=pd.DataFrame(svm_test_prediction)
svm_test_prediction.iloc[:,0].value_counts()

1    1201
0     209
Name: 0, dtype: int64

In [595]:
## Implementing Simple Neural Network

In [596]:
input_dim = training_data_initial.shape[1]
print(input_dim)

14845


In [597]:
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [598]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
            metrics=['accuracy'])

In [599]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 10)                148460    
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 11        
Total params: 148,471
Trainable params: 148,471
Non-trainable params: 0
_________________________________________________________________


In [600]:
history = model.fit(training_data_initial, train_y,
                    epochs=100,
                    verbose=False,
                    validation_data=(dev_data_final, dev_data['label']),
                    batch_size=10)

In [601]:
loss, accuracy = model.evaluate(training_data_initial, train_y, verbose=False)
print("Training Accuracy: {:.2f}".format(accuracy))

Training Accuracy: 1.00


In [602]:
final_prediction_validation=model.predict_classes(dev_data_final)

In [603]:
nn_results = classification_report(final_prediction_validation, dev_data['label'])
print(nn_results)
confusion_matrix(final_prediction_validation,dev_data['label'])

              precision    recall  f1-score   support

           0       0.22      0.58      0.32        19
           1       0.84      0.52      0.64        81

    accuracy                           0.53       100
   macro avg       0.53      0.55      0.48       100
weighted avg       0.72      0.53      0.58       100



array([[11,  8],
       [39, 42]])

In [604]:
final_prediction=model.predict_classes(testing_data_initial)

In [605]:
final_prediction=pd.DataFrame(final_prediction)

In [606]:
final_prediction.iloc[:,0].value_counts()

1    968
0    442
Name: 0, dtype: int64

In [607]:
test_id = {}
for i in range(0,len(final_prediction)):
    key = "test-"+str(i)
    test_id[key] = {}
    label_dict = {}
    label_key = "label"
    label_dict[label_key] = int(final_prediction[0][i])
    test_id[key] = label_dict
with open(r'/Users/varunsai/Desktop/test-output.json', 'w') as outfile:
    json.dump(test_id, outfile)